In [1]:
import sys
sys.path.append('../BPM_extraction')
from BPM import BPM_extractor

import  matplotlib.pyplot as plt 
from pydub import AudioSegment
import numpy as np 
import librosa
import pandas as pd
# import oskars stuffs




/Users/jarlstephansson/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

class sound_C(): 
    def __init__(self, path):
        sound = AudioSegment.from_file(path)
        self.fs = sound.frame_rate
        if sound.channels == 2:
            sound = sound.set_channels(1)
        # normalize: 
        audio = np.array(sound.get_array_of_samples())
        audio = audio / np.max(np.abs(audio))
        self.sound = audio
        
    def shorten(self, start = 0.0, end = 1.1): # in secs
        self.sound = self.sound[int(start* self.fs):int(end*self.fs)]
    
    def give_short(self, start = 0.0, end = 1.1):
        return self.sound[int(start* self.fs):int(end*self.fs)]

    def __len__(self):
        return len(self.sound)

In [ ]:
map_path = "../data/map.csv"
data_path = "../data/KEY-BPM_train-val"

def loop(map_path, data_path): 
    df =pd.read_csv(map_path)
    errors = []

    # here we loop thorugh all the rows in the csv file
    for index, row in df.iterrows():
        file_path = f"{data_path}/{row['song']}.mp3"
        print(file_path)
        sound = sound_C(f"{file_path}")
        sr = sound.fs
        audio = sound.sound
        Extractor = BPM_extractor(sr, novelty = "spectral", tempogram = "fourier", 
                 novelty_w = 1.0, novelty_hop = 0.1,
                 tempo_w = 20.0, tempo_hop = 1.0)
        __,__, __ , bpm = Extractor.get_BPM(audio, plot = False)
        mse = np.abs(row["BPM"] - bpm)
        errors.append(mse)
    
    print(np.array(errors).mean())
        
loop(map_path, data_path)

"data/KEY-BPM_train-val"/ alleswirdgut - Brutalismus 3000.mp3

../data/KEY-BPM_train-val/alleswirdgut - Brutalismus 3000.mp3


FileNotFoundError: [Errno 2] No such file or directory: '../data/KEY-BPM_train-val/alleswirdgut - Brutalismus 3000.mp3'